In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import csv
import matplotlib.pyplot as plt

# keras imports
import keras
from keras import backend as K
backend_keras = keras.backend.backend()
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
print("keras is using", backend_keras,"as the backend")

# import a whole bunch of other packeges
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import keras
import pickle
import cv2
import os

Using TensorFlow backend.


keras is using tensorflow as the backend


In [2]:
def show_image(arr):
    two_d = (np.reshape(arr, (28,28))*255).astype(np.uint8)
    plt.imshow(two_d, cmap='Greys', interpolation='nearest')
    plt.show()

# load the PADDED TRAIN IMAGES
train_images = np.load("../padded_train_images.npy")


Transform and normalize images??

In [3]:
# load the training labels into a dictionary,
# also load it into a list, haven't decided which one is better yet
labels_dic = {}
labels = []
with open('../train_labels.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        try: 
            labels_dic[int(row[0])] = int(row[1])
            labels.append(int(row[1]))
        except: print(row)
            
len(labels_dic)

['ID', 'label']


50000

In [27]:
# normalize images
train_images = np.array(train_images, dtype="float") / 255.0

# reshape the images so that they have 3 channels, 3 identical copies
train_images_3 = [[[[k,k,k] for k in j] for j in i] for i in train_images]
train_images_3 = np.array(train_images_3)    

labels = np.array(labels)
train_labels = []
for i in labels:
    label = np.zeros(10)
    label[i]=1
    train_labels.append(label)

train_labels = np.array(train_labels)
#print(labels[2])

In [28]:
print(train_images.shape)
print(train_images_3.shape)

(50000, 32, 32)
(50000, 32, 32, 3)


In [29]:
print(train_labels.shape,train_images.shape)

(50000, 10) (50000, 32, 32)


## Loading Keras Model

following this guid
https://www.learnopencv.com/keras-tutorial-using-pre-trained-imagenet-models/

and also following this guid
https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

In [71]:
# load the VGG model
vgg_conv = vgg16.VGG16(include_top=False,
                        weights='imagenet',
                        input_shape=(32,32,3))

# load the inception_V3 model
inception_model = inception_v3.InceptionV3(weights='imagenet')

# load the resnet50 model
resnet_model = resnet50.ResNet50(weights='imagenet')

# load the mobilenet model
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

### Freeze the required layers

In [102]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-2]:
    layer.trainable = False
    
# check the trainable status of individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7faca0a81550> False
<keras.layers.convolutional.Conv2D object at 0x7faca0a81898> False
<keras.layers.convolutional.Conv2D object at 0x7facae4f56d8> False
<keras.layers.pooling.MaxPooling2D object at 0x7faca0af25f8> False
<keras.layers.convolutional.Conv2D object at 0x7faca0af2d68> False
<keras.layers.convolutional.Conv2D object at 0x7faca0a2bf60> False
<keras.layers.pooling.MaxPooling2D object at 0x7faca0a05780> False
<keras.layers.convolutional.Conv2D object at 0x7faca0a5ba20> False
<keras.layers.convolutional.Conv2D object at 0x7faca0b62a90> False
<keras.layers.convolutional.Conv2D object at 0x7faca0b4dc50> False
<keras.layers.pooling.MaxPooling2D object at 0x7faca0b4d0f0> False
<keras.layers.convolutional.Conv2D object at 0x7faca0b4d198> False
<keras.layers.convolutional.Conv2D object at 0x7faca0b129e8> False
<keras.layers.convolutional.Conv2D object at 0x7faca0b12eb8> False
<keras.layers.pooling.MaxPooling2D object at 0x7faca0b26b38>

In [103]:
# compile the vgg model
vgg_model.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])

### Create a new model

In [104]:
from keras import models
from keras import layers
from keras import optimizers

# create a model
model = models.Sequential()

# add the vgg convolutional base model
model.add(vgg_conv)

# add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# show a summary of the model. Check the number of trainable params
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                10250     
Total params: 15,250,250
Trainable params: 535,562
Non-trainable params: 14,714,688
_________________________________________________________________


### batchsize

In [105]:
# change batchsize according to ram
train_batchsize = 64
val_batchsize=10

#### train val split

In [106]:
r = 0.8
l = len(train_images_3)
cutoff = int(r*l)
x_train = train_images_3[:cutoff]
y_train = train_labels[:cutoff]
x_val = train_images_3[cutoff:]
y_val = train_labels[cutoff:]

### train the model

In [107]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# train the model
history = model.fit(train_images_3,
                    train_labels,
                    epochs=35,
                    batch_size=32,
                    validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/35
40000/40000 [==============================] - 486s 12ms/step - loss: 2.3026 - acc: 0.0980 - val_loss: 2.3026 - val_acc: 0.0960
Epoch 2/35
40000/40000 [==============================] - 504s 13ms/step - loss: 2.3026 - acc: 0.1000 - val_loss: 2.3026 - val_acc: 0.0960
Epoch 3/35
40000/40000 [==============================] - 494s 12ms/step - loss: 2.3026 - acc: 0.1012 - val_loss: 2.3027 - val_acc: 0.0960
Epoch 4/35
40000/40000 [==============================] - 523s 13ms/step - loss: 2.3026 - acc: 0.1012 - val_loss: 2.3027 - val_acc: 0.0960
Epoch 5/35
40000/40000 [==============================] - 476s 12ms/step - loss: 2.3026 - acc: 0.1012 - val_loss: 2.3027 - val_acc: 0.0960
Epoch 6/35
40000/40000 [==============================] - 522s 13ms/step - loss: 2.3026 - acc: 0.1012 - val_loss: 2.3027 - val_acc: 0.0960
Epoch 7/35
40000/40000 [==============================] - 526s 13ms/step - loss: 2.3026 - acc: 0.1012 - val_loss: 2.

KeyboardInterrupt: 

### save the model

In [ ]:
# save the model
model.save('last2.h5')

### save the model to json
https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [ ]:
from keras.models import model_from_json
import numpy
import os

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_last2.json","w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


### Load model from json

In [ ]:
# load json and create model
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("loaded model from disk")